In [1]:
import gradio as gr
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from datetime import datetime, timedelta
import random
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

class ProphetEngine:
    def __init__(self):
        self.scaler = StandardScaler()
        self.churn_model = RandomForestClassifier(n_estimators=100, random_state=42)
        self.sample_data = self.generate_sample_data()
        self.train_models()

    def generate_sample_data(self):
        """Generate realistic sample MSP client data"""
        np.random.seed(42)
        clients = []

        client_names = [
            "TechCorp Solutions", "DataFlow Systems", "CloudFirst Inc", "SecureNet Co",
            "InnovateTech Ltd", "DigitalEdge Corp", "CyberGuard Systems", "NetOptimize Inc",
            "SmartIT Solutions", "ProActive Tech", "ReliableNet Co", "TechSupport Plus",
            "CloudMaster Systems", "DataSecure Inc", "NetworkPro Ltd"
        ]

        for i, name in enumerate(client_names):
            # Generate realistic metrics with some correlation
            monthly_revenue = np.random.normal(8000, 3000)
            monthly_revenue = max(2000, monthly_revenue)

            # Higher revenue clients tend to have more tickets but better resolution times
            ticket_volume = np.random.poisson(max(5, monthly_revenue / 400))
            avg_resolution_hours = np.random.exponential(2.5) + 1

            # Calculate costs (technician time, software licenses, etc.)
            technician_hours = ticket_volume * avg_resolution_hours
            hourly_rate = 75
            technician_cost = technician_hours * hourly_rate

            software_licenses = monthly_revenue * np.random.uniform(0.15, 0.35)
            hardware_allocation = monthly_revenue * np.random.uniform(0.05, 0.15)

            total_costs = technician_cost + software_licenses + hardware_allocation
            net_profit = monthly_revenue - total_costs
            profit_margin = (net_profit / monthly_revenue) * 100

            # Sentiment score (0-10, higher is better)
            sentiment_score = np.random.normal(7, 1.5)
            sentiment_score = max(1, min(10, sentiment_score))

            # Contract renewal in days
            contract_renewal_days = np.random.randint(30, 365)

            # Service usage metrics
            uptime_percentage = np.random.normal(99.2, 1.2)
            uptime_percentage = max(95, min(100, uptime_percentage))

            alert_frequency = np.random.poisson(15)

            # Calculate churn risk based on multiple factors
            churn_risk = 0
            if profit_margin < 10:
                churn_risk += 30
            if sentiment_score < 5:
                churn_risk += 40
            if avg_resolution_hours > 4:
                churn_risk += 25
            if uptime_percentage < 98:
                churn_risk += 20
            if contract_renewal_days < 60:
                churn_risk += 15

            churn_risk = min(95, max(5, churn_risk + np.random.normal(0, 10)))

            clients.append({
                'client_name': name,
                'monthly_revenue': round(monthly_revenue, 2),
                'ticket_volume': ticket_volume,
                'avg_resolution_hours': round(avg_resolution_hours, 2),
                'technician_cost': round(technician_cost, 2),
                'software_licenses': round(software_licenses, 2),
                'hardware_allocation': round(hardware_allocation, 2),
                'total_costs': round(total_costs, 2),
                'net_profit': round(net_profit, 2),
                'profit_margin': round(profit_margin, 2),
                'sentiment_score': round(sentiment_score, 1),
                'contract_renewal_days': contract_renewal_days,
                'uptime_percentage': round(uptime_percentage, 2),
                'alert_frequency': alert_frequency,
                'churn_risk': round(churn_risk, 1)
            })

        return pd.DataFrame(clients)

    def train_models(self):
        """Train the churn prediction model"""
        features = ['monthly_revenue', 'ticket_volume', 'avg_resolution_hours',
                   'profit_margin', 'sentiment_score', 'uptime_percentage',
                   'alert_frequency', 'contract_renewal_days']

        X = self.sample_data[features]
        # Create binary churn target (high risk = 1, low risk = 0)
        y = (self.sample_data['churn_risk'] > 50).astype(int)

        X_scaled = self.scaler.fit_transform(X)
        self.churn_model.fit(X_scaled, y)

    def get_client_health_dashboard(self):
        """Generate the main client health dashboard"""
        df = self.sample_data.copy()

        # Sort by churn risk
        df_sorted = df.sort_values('churn_risk', ascending=False)

        # Create risk categories
        df_sorted['risk_category'] = pd.cut(df_sorted['churn_risk'],
                                          bins=[0, 30, 60, 100],
                                          labels=['Low Risk', 'Medium Risk', 'High Risk'])

        # Create the visualization
        fig = go.Figure()

        colors = {'High Risk': '#FF4444', 'Medium Risk': '#FFA500', 'Low Risk': '#00AA00'}

        for category in ['High Risk', 'Medium Risk', 'Low Risk']:
            category_data = df_sorted[df_sorted['risk_category'] == category]
            fig.add_trace(go.Scatter(
                x=category_data['monthly_revenue'],
                y=category_data['churn_risk'],
                mode='markers+text',
                marker=dict(
                    size=category_data['profit_margin'].abs() * 0.5 + 10,
                    color=colors[category],
                    opacity=0.7
                ),
                text=category_data['client_name'],
                textposition="top center",
                name=category,
                hovertemplate=
                    "<b>%{text}</b><br>" +
                    "Monthly Revenue: $%{x:,.0f}<br>" +
                    "Churn Risk: %{y:.1f}%<br>" +
                    "Profit Margin: " + category_data['profit_margin'].round(1).astype(str) + "%<br>" +
                    "<extra></extra>"
            ))

        fig.update_layout(
            title="Client Health Dashboard - Churn Risk vs Revenue",
            xaxis_title="Monthly Revenue ($)",
            yaxis_title="Churn Risk (%)",
            height=600,
            showlegend=True
        )

        return fig, df_sorted[['client_name', 'churn_risk', 'monthly_revenue', 'net_profit', 'profit_margin']].head(10)

    def get_profitability_analysis(self):
        """Generate profitability analysis"""
        df = self.sample_data.copy()

        # Categorize clients by profitability
        df['profitability_category'] = pd.cut(df['profit_margin'],
                                            bins=[-float('inf'), 0, 20, float('inf')],
                                            labels=['Vampire Clients', 'Break-even', 'VIP Clients'])

        fig = px.scatter(df,
                        x='monthly_revenue',
                        y='net_profit',
                        size='ticket_volume',
                        color='profitability_category',
                        hover_data=['client_name', 'profit_margin'],
                        title="True Profitability Analysis",
                        color_discrete_map={
                            'Vampire Clients': '#FF4444',
                            'Break-even': '#FFA500',
                            'VIP Clients': '#00AA00'
                        })

        fig.update_layout(height=600)

        # Summary stats
        vampire_clients = df[df['profitability_category'] == 'Vampire Clients']
        vip_clients = df[df['profitability_category'] == 'VIP Clients']

        summary = {
            'Total Vampire Clients': len(vampire_clients),
            'Vampire Revenue Lost': f"${vampire_clients['net_profit'].sum():,.0f}",
            'Total VIP Clients': len(vip_clients),
            'VIP Profit Generated': f"${vip_clients['net_profit'].sum():,.0f}",
            'Average Profit Margin': f"{df['profit_margin'].mean():.1f}%"
        }

        return fig, summary

    def generate_ai_insights(self, client_name):
        """Generate AI-powered insights for a specific client"""
        client_data = self.sample_data[self.sample_data['client_name'] == client_name].iloc[0]

        insights = []
        recommendations = []

        # Churn risk analysis
        if client_data['churn_risk'] > 70:
            insights.append(f"🚨 CRITICAL: {client_name} has a {client_data['churn_risk']:.1f}% churn probability.")
            recommendations.append("Schedule immediate executive review meeting within 7 days")
        elif client_data['churn_risk'] > 40:
            insights.append(f"⚠️ WARNING: {client_name} shows elevated churn risk at {client_data['churn_risk']:.1f}%.")
            recommendations.append("Implement proactive check-in within 2 weeks")

        # Profitability analysis
        if client_data['profit_margin'] < 0:
            insights.append(f"💸 VAMPIRE CLIENT: Losing ${abs(client_data['net_profit']):,.0f}/month ({client_data['profit_margin']:.1f}% margin)")
            recommendations.append("Renegotiate contract terms or implement efficiency improvements")
        elif client_data['profit_margin'] > 30:
            insights.append(f"💎 VIP CLIENT: Generating ${client_data['net_profit']:,.0f}/month ({client_data['profit_margin']:.1f}% margin)")
            recommendations.append("Consider upselling additional services")

        # Operational insights
        if client_data['avg_resolution_hours'] > 3:
            insights.append(f"⏱️ INEFFICIENCY: Average resolution time is {client_data['avg_resolution_hours']:.1f} hours (above optimal)")
            recommendations.append("Investigate recurring issues and implement knowledge base improvements")

        if client_data['uptime_percentage'] < 98:
            insights.append(f"📉 PERFORMANCE ISSUE: Uptime at {client_data['uptime_percentage']:.1f}% (below SLA)")
            recommendations.append("Infrastructure review and upgrade consultation needed")

        # Sentiment analysis
        if client_data['sentiment_score'] < 5:
            insights.append(f"😞 SATISFACTION CONCERN: Client sentiment score is {client_data['sentiment_score']:.1f}/10")
            recommendations.append("Deploy satisfaction survey and implement service improvements")

        return insights, recommendations

    def get_resource_forecast(self):
        """Generate resource and budget forecasting"""
        df = self.sample_data.copy()

        # Simulate next 6 months forecast
        months = ['Current', 'Month +1', 'Month +2', 'Month +3', 'Month +4', 'Month +5', 'Month +6']

        current_tech_hours = df['ticket_volume'].sum() * df['avg_resolution_hours'].mean()
        current_costs = df['total_costs'].sum()

        # Apply growth trends and seasonality
        forecast_data = []
        for i, month in enumerate(months):
            growth_factor = 1 + (i * 0.02)  # 2% monthly growth
            seasonal_factor = 1 + 0.1 * np.sin(i * np.pi / 6)  # Seasonal variation

            forecast_tech_hours = current_tech_hours * growth_factor * seasonal_factor
            forecast_costs = current_costs * growth_factor * seasonal_factor

            forecast_data.append({
                'Month': month,
                'Predicted_Tech_Hours': forecast_tech_hours,
                'Predicted_Costs': forecast_costs,
                'Required_Technicians': np.ceil(forecast_tech_hours / 160)  # 160 hours per month per tech
            })

        forecast_df = pd.DataFrame(forecast_data)

        fig = go.Figure()
        fig.add_trace(go.Scatter(
            x=forecast_df['Month'],
            y=forecast_df['Predicted_Tech_Hours'],
            mode='lines+markers',
            name='Predicted Tech Hours',
            line=dict(color='#1f77b4')
        ))

        fig.add_trace(go.Scatter(
            x=forecast_df['Month'],
            y=forecast_df['Required_Technicians'] * 50,  # Scale for visibility
            mode='lines+markers',
            name='Required Technicians (x50)',
            line=dict(color='#ff7f0e'),
            yaxis='y2'
        ))

        fig.update_layout(
            title="Resource Forecast - Next 6 Months",
            xaxis_title="Time Period",
            yaxis_title="Technician Hours",
            yaxis2=dict(
                title="Required Technicians",
                overlaying='y',
                side='right'
            ),
            height=500
        )

        return fig, forecast_df[['Month', 'Required_Technicians', 'Predicted_Costs']]

# Initialize the Prophet Engine
engine = ProphetEngine()

def create_client_health_tab():
    """Create the client health monitoring interface"""
    with gr.Row():
        with gr.Column(scale=2):
            health_plot = gr.Plot(label="Client Health Dashboard")
        with gr.Column(scale=1):
            risk_table = gr.Dataframe(
                label="Top 10 At-Risk Clients",
                headers=["Client", "Churn Risk %", "Monthly Revenue", "Net Profit", "Profit Margin %"]
            )

    def update_health_dashboard():
        fig, risk_data = engine.get_client_health_dashboard()
        return fig, risk_data

    # Auto-update on tab load
    health_plot.value, risk_table.value = update_health_dashboard()

    refresh_btn = gr.Button("🔄 Refresh Data", variant="primary")
    refresh_btn.click(update_health_dashboard, outputs=[health_plot, risk_table])

    return health_plot, risk_table, refresh_btn

def create_profitability_tab():
    """Create the profitability analysis interface"""
    with gr.Row():
        with gr.Column(scale=2):
            profit_plot = gr.Plot(label="Profitability Analysis")
        with gr.Column(scale=1):
            profit_summary = gr.JSON(label="Profitability Summary")

    def update_profitability():
        fig, summary = engine.get_profitability_analysis()
        return fig, summary

    # Auto-update on tab load
    profit_plot.value, profit_summary.value = update_profitability()

    refresh_profit_btn = gr.Button("🔄 Refresh Analysis", variant="primary")
    refresh_profit_btn.click(update_profitability, outputs=[profit_plot, profit_summary])

    return profit_plot, profit_summary, refresh_profit_btn

def create_ai_insights_tab():
    """Create the AI insights interface"""
    client_dropdown = gr.Dropdown(
        choices=engine.sample_data['client_name'].tolist(),
        label="Select Client for AI Analysis",
        value=engine.sample_data['client_name'].iloc[0]
    )

    with gr.Row():
        with gr.Column():
            insights_output = gr.Markdown(label="🤖 AI-Generated Insights")
        with gr.Column():
            recommendations_output = gr.Markdown(label="💡 Actionable Recommendations")

    def generate_insights(client_name):
        insights, recommendations = engine.generate_ai_insights(client_name)

        insights_md = "### Key Insights\n" + "\n".join([f"- {insight}" for insight in insights])
        recommendations_md = "### Recommended Actions\n" + "\n".join([f"{i+1}. {rec}" for i, rec in enumerate(recommendations)])

        return insights_md, recommendations_md

    analyze_btn = gr.Button("🔍 Generate AI Insights", variant="primary")
    analyze_btn.click(
        generate_insights,
        inputs=[client_dropdown],
        outputs=[insights_output, recommendations_output]
    )

    # Auto-generate for first client
    initial_insights, initial_recs = generate_insights(client_dropdown.value)
    insights_output.value = initial_insights
    recommendations_output.value = initial_recs

    return client_dropdown, insights_output, recommendations_output, analyze_btn

def create_forecast_tab():
    """Create the resource forecasting interface"""
    with gr.Row():
        with gr.Column(scale=2):
            forecast_plot = gr.Plot(label="Resource Forecast")
        with gr.Column(scale=1):
            forecast_table = gr.Dataframe(label="Staffing Requirements")

    def update_forecast():
        fig, forecast_data = engine.get_resource_forecast()
        return fig, forecast_data

    # Auto-update on tab load
    forecast_plot.value, forecast_table.value = update_forecast()

    refresh_forecast_btn = gr.Button("🔄 Update Forecast", variant="primary")
    refresh_forecast_btn.click(update_forecast, outputs=[forecast_plot, forecast_table])

    return forecast_plot, forecast_table, refresh_forecast_btn

# Create the main Gradio interface
def create_prophet_interface():
    with gr.Blocks(title="Prophet - MSP Intelligence Engine", theme=gr.themes.Soft()) as demo:
        gr.Markdown("""
        # 🔮 Prophet - AI-Powered MSP Intelligence Engine

        **Predictive Client Health & Profitability Analytics for Managed Service Providers**

        Transform your MSP operations from reactive to predictive with AI-driven insights that identify at-risk clients,
        optimize profitability, and forecast resource needs before problems impact your bottom line.
        """)

        with gr.Tabs():
            with gr.Tab("📊 Client Health Monitor"):
                gr.Markdown("### Real-time client health scoring and churn risk prediction")
                create_client_health_tab()

            with gr.Tab("💰 Profitability Analysis"):
                gr.Markdown("### Identify VIP clients and vampire clients draining resources")
                create_profitability_tab()

            with gr.Tab("🤖 AI Insights"):
                gr.Markdown("### Get AI-powered recommendations for specific clients")
                create_ai_insights_tab()

            with gr.Tab("📈 Resource Forecast"):
                gr.Markdown("### Predict future staffing and budget requirements")
                create_forecast_tab()

            with gr.Tab("ℹ️ About Prophet"):
                gr.Markdown("""
                ## About Prophet Engine

                **Prophet** is an AI-powered predictive analytics platform specifically designed for Managed Service Providers (MSPs).

                ### Key Features:

                1. **Predictive Client Health Scoring** - Uses machine learning to predict client churn risk weeks or months in advance
                2. **True Profitability Analysis** - Reveals hidden costs and identifies which clients are actually profitable
                3. **AI-Generated Actionable Insights** - Natural language recommendations powered by advanced AI
                4. **Resource & Budget Forecasting** - Predicts future technician workload and budget requirements

                ### Technology Stack:
                - **Frontend**: Gradio for rapid prototyping and deployment
                - **AI/ML**: Scikit-learn for predictive modeling, integrated with cloud AI services
                - **Visualization**: Plotly for interactive dashboards
                - **Data Processing**: Pandas for data manipulation and analysis

                ### Benefits for MSPs:
                - **Increase Revenue**: Identify upselling opportunities and optimize pricing
                - **Reduce Churn**: Proactively address client issues before they escalate
                - **Improve Efficiency**: Allocate resources based on predictive insights
                - **Strategic Growth**: Make data-driven decisions for business expansion

                *This is a prototype demonstrating the core functionality of the Prophet system.*
                """)

        gr.Markdown("""
        ---
        **Prophet Engine** - Transforming MSP Operations with Predictive AI
        *Built for the AWS MSP Challenge 2024*
        """)

    return demo

# Create and launch the interface
if __name__ == "__main__":
    demo = create_prophet_interface()
    demo.launch(
        server_name="0.0.0.0",
        server_port=7860,
        share=True,
        show_error=True
    )

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ce00aeb59e8ffd0ef7.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
